Connect to MongoDB Atlas
------------------------

In [1]:
import pymongo, pprint

In [2]:
atlas_URI = open("atlas_URI.txt").read().strip()
client = pymongo.MongoClient(atlas_URI)

List Indexes
-----------

In [3]:
# get list of indexes on movies collection
pprint.pprint(client.mflix.movies.index_information())

{'_id_': {'key': [('_id', 1)], 'ns': 'mflix.movies', 'v': 2},
 'genres_1_tomatoes.viewer.numReviews_-1': {'key': [('genres', 1),
                                                    ('tomatoes.viewer.numReviews',
                                                     -1)],
                                            'ns': 'mflix.movies',
                                            'v': 2},
 'title_text_cast_text_directors_text': {'default_language': 'english',
                                         'key': [('_fts', 'text'),
                                                 ('_ftsx', 1)],
                                         'language_override': 'language',
                                         'ns': 'mflix.movies',
                                         'textIndexVersion': 3,
                                         'v': 2,
                                         'weights': {'cast': 1,
                                                     'directors': 1,
                        

In [6]:
pprint.pprint(client.mflix.movies.find_one())

{'_id': ObjectId('573a1390f29313caabcd4132'),
 'cast': ['Carmencita'],
 'countries': ['USA'],
 'directors': ['William K.L. Dickson'],
 'fullplot': 'Performing on what looks like a small wooden stage, wearing a '
             'dress with a hoop skirt and white high-heeled pumps, Carmencita '
             'does a dance with kicks and twirls, a smile always on her face.',
 'genres': ['Documentary', 'Short'],
 'imdb': {'id': 1, 'rating': 5.9, 'votes': 1032},
 'lastupdated': '2015-08-26 00:03:45.040000000',
 'plot': 'Performing on what looks like a small wooden stage, wearing a dress '
         'with a hoop skirt and white high-heeled pumps, Carmencita does a '
         'dance with kicks and twirls, a smile always on her face.',
 'poster': 'http://ia.media-imdb.com/images/M/MV5BMjAzNDEwMzk3OV5BMl5BanBnXkFtZTcwOTk4OTM5Ng@@._V1_SX300.jpg',
 'rated': 'NOT RATED',
 'runtime': 1,
 'title': 'Carmencita',
 'type': 'movie',
 'year': 1894}


Explain a Query
--------------

In [7]:
explain = {
    "explain": {
        "find": "movies",
        "filter": {
            "tomatoes.viewer.numReviews": {"$gt": 10}
        },
    },
    "verbosity": "executionStats"
}
client.mflix.command(explain)

{'$clusterTime': {'clusterTime': Timestamp(1547956437, 5),
  'signature': {'hash': b')\xf2}\xd5\xd5\x95\x88\xbe\xca\x1d\x06\x84\xba\x1d\x1d\x00t%,j',
   'keyId': 6615991365337612289}},
 'executionStats': {'executionStages': {'advanced': 35519,
   'alreadyHasObj': 0,
   'docsExamined': 35519,
   'executionTimeMillisEstimate': 250,
   'inputStage': {'advanced': 35519,
    'direction': 'forward',
    'dupsDropped': 0,
    'dupsTested': 0,
    'executionTimeMillisEstimate': 20,
    'indexBounds': {'tomatoes.viewer.numReviews': ['[inf.0, 10)']},
    'indexName': 'tomatoes.viewer.numReviews_-1',
    'indexVersion': 2,
    'invalidates': 0,
    'isEOF': 1,
    'isMultiKey': False,
    'isPartial': False,
    'isSparse': False,
    'isUnique': False,
    'keyPattern': {'tomatoes.viewer.numReviews': -1},
    'keysExamined': 35519,
    'multiKeyPaths': {'tomatoes.viewer.numReviews': []},
    'nReturned': 35519,
    'needTime': 0,
    'needYield': 0,
    'restoreState': 277,
    'saveState': 277,

Text Search vs Exact Match
-------------------------

In [8]:
filters = {"title": "Titanic"}
for m in client.mflix.movies.find(filters):
    pprint.pprint(m['title'])

'Titanic'
'Titanic'
'Titanic'


In [9]:
filters = { "$text": { 
    "$search": "titanic"
}} 
for m in client.mflix.movies.find(filters):
    pprint.pprint(m['title'])
    pprint.pprint(m['cast'])
    pprint.pprint(m.get('directors', ""))
    pprint.pprint("======")

'Titanic'
['Leonardo DiCaprio', 'Kate Winslet', 'Billy Zane', 'Kathy Bates']
['James Cameron']
'======'
'Titanic'
['Peter Gallagher',
 'George C. Scott',
 'Catherine Zeta-Jones',
 'Eva Marie Saint']
''
'======'
'Titanic'
['Clifton Webb', 'Barbara Stanwyck', 'Robert Wagner', 'Audrey Dalton']
['Jean Negulesco']
'======'
'Drain the Titanic'
['Brad Cartner']
''
'======'
'Attack on Titan'
['Haruma Miura', 'Kiko Mizuhara', 'Kanata Hong�', 'Satomi Ishihara']
['Shinji Higuchi']
'======'
'Wrath of the Titans'
['Sam Worthington', 'Liam Neeson', 'Ralph Fiennes', '�dgar Ram�rez']
['Jonathan Liebesman']
'======'
'Titanic II'
['Shane Van Dyke', 'Marie Westbrook', 'Bruce Davison', 'Brooke Burns']
['Shane Van Dyke']
'======'
'Clash of the Titans'
['Sam Worthington', 'Liam Neeson', 'Ralph Fiennes', 'Jason Flemyng']
['Louis Leterrier']
'======'
'Remember the Titans'
['Denzel Washington', 'Will Patton', 'Wood Harris', 'Ryan Hurst']
['Boaz Yakin']
'======'
'The Chambermaid on the Titanic'
['Olivier Martin

Create an Index
--------------

In [10]:
client.mflix.movies.create_index([("countries", pymongo.ASCENDING)])

'countries_1'